In [1]:
using EISAnalysis

ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


In [2]:
get_params(a::Resistor) = a.R
get_params(a::Capacitor) = a.C
get_params(a::Inductor) = a.L
get_params(a::CPE) = (a.Q,a.n)
get_params(a::Warburg) = (a.A,a.B)

get_params (generic function with 5 methods)

In [ ]:
#let's try to rebuild the following circuit
circuit = r-(r-wo)/q-(r-ws)/q #- (r-ws)/c
# circuit = r-r/c-r/c
println(circuit.elements)
println(vcat(undef,circuit.operators))
println(vcat(undef,circuit.order))
println(circuit.subcircuits)

[:r, :r, :wo, :q, :r, :ws, :q]
Any[UndefInitializer(), -, -, /, -, -, /]
[0, 3, 1, 2, 6, 4, 5]
[1, 1, 1, 1, 2, 2, 2]


In [ ]:
function rebuild(subelements,suboperators,suborder)
    suborder = suborder[2:end]
    suboperators = suboperators[2:end]
    #sorted_index
    si = sortperm(suborder)
    #initialize subcircuit
    if suboperators[si[1]]==-
        subcircuit = eval(:( $(subelements[si[1]]) - $(subelements[si[2]]) ) )
    elseif suboperators[si[1]]==/
        subcircuit = eval(:( $(subelements[si[1]]) / $(subelements[si[2]]) ) )  
    #rest of the subcircuit
    for i in si[2:end]
        if suboperators[si[i]]==-
            subcircuit = eval(:( $subcircuit - $(subelements[si[i+1]]) ) )
        elseif suboperators[si[i]]==/
            subcircuit = eval(:( $subcircuit / $(subelements[si[i+1]]) ) )  


end


# function rebuild(circuit)
#     fullcircuit = undef
#     operators = vcat(undef,circuit.operators)
#     order = vcat(undef,circuit.order)
#     for i in 1:maximum(circuit.subcircuits)
#         sub_i = findall(isequal(i),circuit.subcircuits)
#         subelements = circuit.elements[sub_i]
#         suboperators = circuit.operators[sub_i]
#         suborder = order[sub_i]
#     end
# end
rebuild(circuit)

Any[UndefInitializer(), -, -, /, -, -, -, /]
Any[UndefInitializer(), -, -, /, -, -, -, /]


In [31]:
order = vcat(0,circuit.order)

circuit.elements[sortperm(order[2:end]).+1]

6-element Vector{Symbol}:
 :wo
 :q
 :r
 :ws
 :q
 :r

In [ ]:
function rebuild_circuit(circuit::Circuit)

    subcircuits = Vector(undef,maximum(circuit.subcircuits))
    subelements = [circuit.elements[findall(isequal(i),circuit.subcircuits)] for i in eachindex(subcircuits)]
    



    sis = [sortperm(circuit.order[findall(isequal(i),circuit.subcircuits)]) for i in eachindex(subcircuits)]
    for i in eachindex(subcircuits)
        #generate the sorted indices
        si = sis[i]
        if i ==1
            subelements = circuit.elements[1:length(si)+1]
        else
            subelements = circuit.elements[length(sis[i-1])+1:length(si)+1]
        end
        #initializing the rebuilt subcircuit
        if circuit.operators[si[1]]==-
            subcircuits[i] = eval(:($(circuit.elements[si[1]])-$(circuit.elements[si[1+1]])))
        elseif circuit.operators[si[1]]==/
            subcircuits[i] = eval(:($(circuit.elements[si[1]])/$(circuit.elements[si[1+1]])))   
        end
        for j in 2:length(si)-1
            if circuit.operators[j]==-
                subcircuits[i] = eval(:($(subcircuits[i])-$(circuit.elements[si[j+1]])))
            elseif circuit.operators[j]==/
                subcircuits[i] = eval(:($(subcircuits[i])/$(circuit.elements[si[j+1]])))
            end
        end
    end
    fullcircuit = subcircuits[1]
    for subcircuit in subcircuits
        fullcircuit = fullcircuit-subcircuit
    end
    return fullcircuit
end 

rebuild_circuit (generic function with 1 method)

In [14]:
@time fullcircuit = rebuild_circuit(circuit)
println(fullcircuit.elements)
println(fullcircuit.operators)
println(fullcircuit.order)
println(fullcircuit.subcircuits)

  0.022575 seconds (15.82 k allocations: 766.648 KiB, 97.75% compilation time)
[:r, :wo, :r, :q, :r, :wo, :r, :q, :r, :r, :r, :r]
Function[-, -, /, -, -, -, /, -, -, -, -]
[1, 2, 3, 4, 5, 6, 7, 8]
[1, 1, 1, 2, 2, 2, 3, 4]


In [15]:
fullcircuit.Z-circuit.Z

57-element Vector{ComplexF64}:
    2.9977639320225 + 0.0022460679774997895im
 2.9973641932028556 + 0.0026497017520868155im
 2.9968929936201523 + 0.003126313357136844im
  2.996337558331332 + 0.0036892686266212045im
 2.9956828286342123 + 0.004354447302990555im
 2.9949110538029817 + 0.005140740943811499im
 2.9940013098388016 + 0.00607065872849844im
 2.9929289321881347 + 0.007171067811865474im
 2.9916648470488747 + 0.00847410250056286im
 2.9901747831350054 + 0.010018286637882325im
                    ⋮
 1.3333844083632869 + 0.008954237535419907im
  1.333359787922614 + 0.006444336721358468im
 1.3333470355300534 + 0.004637934181971559im
 1.3333404303807441 + 0.003337868713970238im
 1.3333370092389245 + 0.0024022214846581846im
 1.3333352372614797 + 0.0017288463216118544im
 1.3333343194687473 + 0.001244226649760094im
 1.3333338440997977 + 0.0008954523630109179im
 1.3333335978838932 + 0.0006444443370128283im

In [4]:
a = [2,4,6,8]

4-element Vector{Int64}:
 2
 4
 6
 8

In [5]:
vcat(Vector(undef,1),a)

5-element Vector{Any}:
 #undef
   2
   4
   6
   8